In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
import os
from glob import glob

import yaml

import numpy as np
import xarray as xr
import pandas as pd

import config
import util

In [8]:
cluster, client = util.get_ClusterClient()
cluster.scale(12)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/asm/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/asm/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.205.13:38543,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/asm/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [9]:
USER = os.environ['USER']
dirout = f'/glade/work/{USER}/sno-analysis/sno-flux-products'

flux_files = sorted(glob(f'{dirout}/*.nc'))

convert = dict(
    CO2={'factor': 1e-15 * 86400.0 * 365.25, 'units': 'Pmol/yr'},
    O2={'factor': 1e-15 * 86400.0 * 365.25, 'units': 'Pmol/yr'},
    APO={'factor': 1e-15 * 86400.0 * 365.25, 'units': 'Pmol/yr'},    
    N2={'factor': 1e-15 * 86400.0 * 365.25, 'units': 'Pmol/yr'},
)

index = []
columns = []
data = []
for n, f in enumerate(flux_files):
    v = os.path.basename(f).split('.')[0]
    product = os.path.basename(f).split('.')[1]
    print(f'{v}: {product}')
          
    constituent = [c for c in ['CO2', 'O2', 'N2', 'APO'] if c in v][0]
    scale_by = convert[constituent]['factor']
    units = convert[constituent]['units']
    columns.append(f'{v} ({product}) [{units}]')
    
    with xr.open_dataset(f, chunks={'time': 100}) as ds:
        total = (ds[v] * ds.area).sum(['lat', 'lon'])
        total *= scale_by
        data.append(total.values)
        if n == 0:
            index = total.time.values
        else:
            assert (index == total.time.values).all()

df = pd.DataFrame(data=np.array(data).T, index=index, columns=columns)
df            

SFAPO_OCN: carboscope
SFCO2_FF: GCP-GridFED
SFCO2_FF: OCO2-MIP
SFCO2_OCN: MPI-SOM-FFN
SFCO2_OCN: cesm_fosi_smyle
SFN2_OCN: cesm_fosi_smyle
SFN2_OCN: era5_shf
SFO2_FF: GCP-GridFED
SFO2_OCN: cesm_fosi_smyle
SFO2_OCN: gk2001_R2016


,SFAPO_OCN (carboscope) [Pmol/yr],SFCO2_FF (GCP-GridFED) [Pmol/yr],SFCO2_FF (OCO2-MIP) [Pmol/yr],SFCO2_OCN (MPI-SOM-FFN) [Pmol/yr],SFCO2_OCN (cesm_fosi_smyle) [Pmol/yr],SFN2_OCN (cesm_fosi_smyle) [Pmol/yr],SFN2_OCN (era5_shf) [Pmol/yr],SFO2_FF (GCP-GridFED) [Pmol/yr],SFO2_OCN (cesm_fosi_smyle) [Pmol/yr],SFO2_OCN (gk2001_R2016) [Pmol/yr]
1986-01-01 12:00:00,0.000000,0.528720,0.000000,-0.130405,-0.218934,0.033259,0.476587,-0.733670,0.144992,0.653724
1986-01-02 12:00:00,0.000000,0.529393,0.000000,-0.130006,-0.213774,0.029607,0.473220,-0.734536,0.231176,0.653285
1986-01-03 12:00:00,0.000000,0.530066,0.000000,-0.129607,-0.205181,0.025954,0.469853,-0.735402,0.382042,0.652845
1986-01-04 12:00:00,0.000000,0.530738,0.000000,-0.129208,-0.245267,0.022302,0.466486,-0.736268,0.056199,0.652405
1986-01-05 12:00:00,0.000000,0.531411,0.000000,-0.128809,-0.242441,0.018650,0.463120,-0.737134,-0.011542,0.651965
...,...,...,...,...,...,...,...,...,...,...
2020-12-27 12:00:00,1.423361,0.898174,0.842327,-0.265362,-0.381446,0.076151,0.408909,-1.241463,0.233372,0.655923
2020-12-28 12:00:00,1.457859,0.899931,0.899796,-0.266242,-0.343846,0.076168,0.410834,-1.244030,0.391277,0.655483
2020-12-29 12:00:00,1.489718,0.901688,0.903971,-0.267122,-0.353151,0.076185,0.412759,-1.246597,0.406275,0.655044
2020-12-30 12:00:00,1.518499,0.903445,0.903898,-0.268002,-0.436713,0.076202,0.414685,-1.249164,0.237832,0.654604


In [10]:
df.to_csv(f'{dirout}/daily-totals.csv')

In [11]:
tracer_flux_mapping = yaml.safe_load(f"""
apo_jena:
  constituent: O2
  flux_file: SFAPO_OCN.carboscope.apo99X_v2021.0.9x1.25.19860101-20201231.nc
  flux_variable: SFAPO_OCN
co2_cesm:
  constituent: CO2
  flux_file: SFCO2_OCN.cesm_fosi_smyle.0.9x1.25.19860101-20201231.nc
  flux_variable: SFCO2_OCN
co2_gridfed:
  constituent: CO2
  flux_file: SFCO2_FF.GCP-GridFED.v2021.3.0.9x1.25.19860101-20201231.nc
  flux_variable: SFCO2_FF
co2_oco2mip:
  constituent: CO2
  flux_file: SFCO2_FF.OCO2-MIP.v2020.1.0.9x1.25.19860101-20201231.nc
  flux_variable: SFCO2_FF
co2_somffn:
  constituent: CO2
  flux_file: SFCO2_OCN.MPI-SOM-FFN.0.9x1.25.19860101-20201231.nc
  flux_variable: SFCO2_OCN
n2_cesm:
  constituent: N2
  flux_file: SFN2_OCN.cesm_fosi_smyle.0.9x1.25.19860101-20201231.nc
  flux_variable: SFN2_OCN
n2_era:
  constituent: N2
  flux_file: SFN2_OCN.era5_shf.0.9x1.25.19860101-20201231.nc
  flux_variable: SFN2_OCN
o2_cesm:
  constituent: O2
  flux_file: SFO2_OCN.cesm_fosi_smyle.0.9x1.25.19860101-20201231.nc
  flux_variable: SFO2_OCN
o2_gk01r16:
  constituent: O2
  flux_file: SFO2_OCN.gk2001_R2016.0.9x1.25.repeat_monclim.19860101-20201231.nc
  flux_variable: SFO2_OCN
o2_gridfed:
  constituent: O2
  flux_file: SFO2_FF.GCP-GridFED.v2021.3.0.9x1.25.19860101-20201231.nc
  flux_variable: SFO2_FF
""")

tracer_flux_mapping

{'apo_jena': {'constituent': 'O2',
  'flux_file': 'SFAPO_OCN.carboscope.apo99X_v2021.0.9x1.25.19860101-20201231.nc',
  'flux_variable': 'SFAPO_OCN'},
 'co2_cesm': {'constituent': 'CO2',
  'flux_file': 'SFCO2_OCN.cesm_fosi_smyle.0.9x1.25.19860101-20201231.nc',
  'flux_variable': 'SFCO2_OCN'},
 'co2_gridfed': {'constituent': 'CO2',
  'flux_file': 'SFCO2_FF.GCP-GridFED.v2021.3.0.9x1.25.19860101-20201231.nc',
  'flux_variable': 'SFCO2_FF'},
 'co2_oco2mip': {'constituent': 'CO2',
  'flux_file': 'SFCO2_FF.OCO2-MIP.v2020.1.0.9x1.25.19860101-20201231.nc',
  'flux_variable': 'SFCO2_FF'},
 'co2_somffn': {'constituent': 'CO2',
  'flux_file': 'SFCO2_OCN.MPI-SOM-FFN.0.9x1.25.19860101-20201231.nc',
  'flux_variable': 'SFCO2_OCN'},
 'n2_cesm': {'constituent': 'N2',
  'flux_file': 'SFN2_OCN.cesm_fosi_smyle.0.9x1.25.19860101-20201231.nc',
  'flux_variable': 'SFN2_OCN'},
 'n2_era': {'constituent': 'N2',
  'flux_file': 'SFN2_OCN.era5_shf.0.9x1.25.19860101-20201231.nc',
  'flux_variable': 'SFN2_OCN'},
 'o